In [2]:
# Import libraries
from functools import reduce
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, exp
from pandas_datareader.data import DataReader
import requests_cache
%matplotlib inline

In [62]:
# Download data from pandas datareader -> AAPL.O, IBM.N, GOOG.O, BP.N, XOM.N, COST.O, GS.N
# Start: 01/01/2016
# End: 31/12/2016
# Create a dictionary: Key -> Ticker, Value -> Historical data from Start_Date to End_Date

session = requests_cache.CachedSession(cache_name='cache',backend='sqlite')
session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
                   'Accept': 'application/json;charset=utf-8'}

def get_data(tickers, start_date="2016-01-01", end_date="2016-12-31"):
    stocks = dict()
    for ticker in tickers:
        s = DataReader(ticker, 'yahoo', start_date, end_date, session=session)
        s.insert(0, "Ticker", ticker)
        s['Prev Close'] = s['Adj Close'].shift(1)
        s['log_return'] = np.log(s['Adj Close']/s['Prev Close'])
        s['perc_return'] = (s['Adj Close']/s['Prev Close']) - 1
        stocks[ticker] = s
    return stocks

# Initialize the dataset with historical data
stocks = ["AAPL", "IBM", "GOOG", "BP", "XOM", "COST", "GS"]
portfolio = get_data(stocks)

## Question 4a. Using historical daily returns, calculate VaR95% and CVaR95% of portfolio as of 2016/12/31

In [64]:
# Set weights of stocks in portfolio — ["AAPL", "IBM", "GOOG", "BP", "XOM", "COST", "GS"]
weights = np.array([0.15, 0.2, 0.2, 0.15,0.1,0.15,0.05])

# Create column "Previous Adj Close" by shifting "Adj Close" down 1
# Calculate percentage returns by taking (Adj Close / Previous Adj Close) - 1

for stock in stocks:
    portfolio[stock]['Previous Adj Close'] = portfolio[stock]['Adj Close'].shift(1)
    portfolio[stock]['Perc_return'] = (portfolio[stock]['Adj Close'] / portfolio[stock]['Previous Adj Close']) - 1

# Drop na rows, in this case, 2015-12-31 entry

'''portfolio['AAPL'] = portfolio['AAPL'].dropna()
portfolio['IBM'] = portfolio['IBM'].dropna()
portfolio['GOOG'] = portfolio['GOOG'].dropna()
portfolio['BP'] = portfolio['BP'].dropna()
portfolio['XOM'] = portfolio['XOM'].dropna()
portfolio['COST'] = portfolio['COST'].dropna()
portfolio['GS'] = portfolio['GS'].dropna()'''


portfolio['AAPL'].head(3)

,Ticker,High,Low,Open,Close,Volume,Adj Close,Prev Close,log_return,perc_return,Previous Adj Close,Perc_return
Date,,,,,,,,,,,,
2016-01-04,AAPL,26.342501,25.500000,25.652500,26.337500,270597600.0,24.286833,24.266081,0.000855,0.000855,24.266081,0.000855
2016-01-05,AAPL,26.462500,25.602501,26.437500,25.677500,223164000.0,23.678219,24.286833,-0.025379,-0.025059,24.286833,-0.025059
2016-01-06,AAPL,25.592501,24.967501,25.139999,25.174999,273829600.0,23.214844,23.678219,-0.019764,-0.019570,23.678219,-0.019570


In [93]:
# Create a portfolio dataframe to store perc_return, log_return
portfolio_df = pd.DataFrame()
portfolio_df.insert(0, "Date", portfolio['AAPL'].index)
portfolio_df = portfolio_df.set_index("Date")

In [95]:
dates = portfolio_df.index.values
for d in dates:
    d_perc_return = []
    for ticker in stocks:
        d_perc_return.append(portfolio[ticker].loc[d, 'Perc_return'])
    d_perc_return = np.array(d_perc_return)
    portfolio_df.loc[d, "Portfolio_perc_return"] = np.sum(d_perc_return)


In [110]:
# Convert percentage returns to log returns
portfolio_df['Portfolio_log_return'] = np.log(portfolio_df['Portfolio_perc_return'] + 1)

# Using historical data approach to find 95% VaR -> Find the 5th Percentile worst (log) return
VaR_95 = -1* (portfolio_df['Portfolio_log_return'].quantile([0.05]).values[0])
print(f"The VaR95% of portfolio as of 2016/12/31 is {VaR_95}")

The VaR95% of portfolio as of 2016/12/31 is 0.10586921951621035


## Q4ai. The VaR95% of portfolio as of 2016/12/31 is 0.10586921951621035 or 10.59% (4 sig. fig.) log return loss for 95% confidence.

### To find the CVaR95% or expected shortfall at 95% using Historical Data:
### 1. Find sum of loss returns larger than VaR95%
### 2. Divide sum by number of days * alpha, where alpha = 0.05%


In [114]:
loss_return = 0
for d in dates:
    if portfolio_df.loc[d, 'Portfolio_log_return'] < (-1 * VaR_95):
        loss_return += portfolio_df.loc[d, 'Portfolio_log_return']

CVaR_95 = -1*(loss_return/(len(dates) * 0.05))
print(f"The CVaR95% of portfolio as of 2016/12/31 is {CVaR_95}")


The CVaR95% of portfolio as of 2016/12/31 is 0.17560943012814598


## Q4aii. The CVaR95% or Expected Shortfall of portfolio as of 2016/12/31 is 0.17560943012814598 or 17.56% (4 sig fig) log return loss for 95% confidence.
---